# Create Agent

### Initializing Project Client

In [1]:
import os, json
import pandas as pd
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str="eastus.api.azureml.ms;fac34303-435d-4486-8c3f-7094d82a0b60;rg-anksing-5269_ai;anksing-evaluate-project", # os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant for bug bash"

# Adding Tools to be used by Agent 
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

### Create Agent

In [2]:
agent = project_client.agents.create_agent(
    model="gpt-4o-mini", # os.environ["MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_4dHhqqJcm7A0xN1ifgM49odl


### Create Thread

In [3]:
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_pi0NvyqngYeTnIay3xI9j4vq


## Conversation with Agent
Use below cells to have conversation with the agent
- `Create Message[1]`
- `Execute[2]`

### Create Message[1]

In [4]:
# Create message to thread

MESSAGE = "Can you send me an email with weather information for Seattle?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_Lc6vSxjgP58vNYjKlenNZ9Nn


### Execute[2]

In [5]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Sending email to you@example.com...
Subject: Weather Information for Seattle
Body:
Weather data not available for this location.
Run finished with status: RunStatus.COMPLETED
Run ID: run_er847tIPYcYO94i2C1vkfOeZ


### List Messages

In [6]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you send me an email with weather information for Seattle?
----------------------------------------
Role: MessageRole.AGENT
Content: I have sent you an email with the weather information for Seattle. The weather data was not available for this location.
----------------------------------------


# Evaluate

### Get data from agent

In [7]:
import json
from azure.ai.evaluation import AIAgentConverter

# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id

converted_data = converter.convert(thread_id, run_id)
print(json.dumps(converted_data, indent=4))

{
    "query": [
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "createdAt": "2025-03-19T03:27:20Z",
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Can you send me an email with weather information for Seattle?"
                }
            ]
        }
    ],
    "response": [
        {
            "createdAt": "2025-03-19T03:27:23Z",
            "run_id": "run_er847tIPYcYO94i2C1vkfOeZ",
            "role": "assistant",
            "content": [
                {
                    "type": "tool_call",
                    "tool_call": {
                        "id": "call_WfANEHcwQ5E6nsDXWMzsoCXM",
                        "type": "function",
                        "function": {
                            "name": "fetch_weather",
                            "arguments": {
                                "location": "Se

In [8]:
evaluation_data = []
run_ids = converter._list_run_ids_chronological(thread_id=thread.id)

for run_id in run_ids:
    run_data = converter.convert(thread_id, run_id)
    evaluation_data.append(run_data)

# write to file
file_path = os.path.join(os.getcwd(), "evaluation_data.jsonl")
pd.DataFrame(evaluation_data).to_json(
    file_path, orient="records", lines=True
)

### Setting up evaluator

In [ ]:
from azure.ai.evaluation import ToolCallAccuracyEvaluator , AzureOpenAIModelConfiguration, IntentResolutionEvaluator, TaskAdherenceEvaluator
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint="<aoai-endpoint>",
    api_key="<aoai-key>",
    api_version="<aoai-api-version>",
    azure_deployment="<aoai-deployment-name>",
)

tool_call_accuracy = ToolCallAccuracyEvaluator(model_config=model_config)
intent_resolution = IntentResolutionEvaluator(model_config=model_config)
task_adherence = TaskAdherenceEvaluator(model_config=model_config)

### Run Evaluator

In [ ]:
from azure.ai.evaluation import evaluate

response = evaluate(
    data=file_path,
    evaluators={
        "tool_call_accuracy": tool_call_accuracy,
        "intent_resolution": intent_resolution,
        "task_adherence": task_adherence,
    },
    azure_ai_project={
        "subscription_id": "<subscription_id>",
        "project_name": "<project_name>",
        "resource_group_name": "<resource_group_name>",
    }
)
pprint(f'AI Foundary URL: {response.get("studio_url")}')

[2025-03-18 20:27:45 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-03-18 20:27:45 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-03-18 20:27:45 -0700][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-03-18 20:27:45 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_law6c8nn_20250318_202744_924290, log path: C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_law6c8nn_20250318_202744_924290\logs.txt
[2025-03-18 20:27:45 -0700][promptflow._sdk._orchestrator

2025-03-18 20:27:45 -0700   54992 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-03-18 20:27:50 -0700   54992 execution.bulk     INFO     Finished 1 / 1 lines.
2025-03-18 20:27:50 -0700   54992 execution.bulk     INFO     Average execution time for completed lines: 4.4 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ro9lt21p_20250318_202744_924290"
Run status: "Completed"
Start time: "2025-03-18 20:27:44.917561-07:00"
Duration: "0:00:05.959586"
Output path: "C:\Users\anksing\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ro9lt21p_20250318_202744_924290"

2025-03-18 20:27:53 -0700   54992 execution.bulk     INFO     Finished 1 / 1 lines.
2025-03-18 20:27:53 -0700   54992 execution.bulk     INFO     Average execution time for completed lines: 7.57 seconds. Estimated

In [11]:
pprint(response)

{'metrics': {'intent_resolution.intent_resolution': 4.0,
             'intent_resolution.intent_resolution_threshold': 5.0,
             'task_adherence.task_adherence': 3.0,
             'task_adherence.task_adherence_threshold': 5.0,
             'tool_call_accuracy.tool_call_accuracy': 1.0},
 'rows': [{'inputs.query': [{'content': 'You are a helpful assistant',
                             'role': 'system'},
                            {'content': [{'text': 'Can you send me an email '
                                                  'with weather information '
                                                  'for Seattle?',
                                          'type': 'text'}],
                             'createdAt': '2025-03-19T03:27:20Z',
                             'role': 'user'}],
           'inputs.response': [{'content': [{'tool_call': {'function': {'arguments': {'location': 'Seattle'},
                                                                        'name': 